# Crawler (ETF.com)

### Total 28 ETF:
'PZD', 'PXI', 'PXE', 'PXJ', 'PBD', 'RYE', 'PSCE', 'TAN', 'PBW', 'XLE', 'DIG', 'DUG', 'DDG', 'TPYP', 'KOL', 'GEX', 'YMLI', 'YMLP', 'CRAK', 'OIH', 'FRAK', 'NLR', 'ICLN', 'IXC', 'FILL', 'IYE', 'IEO', 'IEZ'
### We've downloaded 27 of total ETF as excel file (see the NAV folder), except for 'TPYP' (dataframe completed, no more need for processing)

In [243]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import time
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [244]:
def Locate_by_linkname(name, text):
    driver = webdriver.Chrome('C:/Users/smile/Documents/GitHub/chromedriver.exe')
    driver.get("https://ETF.com/" + name + "#overview")
    driver.find_element_by_link_text("Fund Home Page").click()
    driver.find_element_by_link_text(text).click()
    time.sleep(12)
    driver.quit()
    
def Locate_by_ID(name, text):
    driver = webdriver.Chrome('C:/Users/smile/Documents/GitHub/chromedriver.exe')
    driver.get("https://ETF.com/" + name + "#overview")
    driver.find_element_by_link_text("Fund Home Page").click()
    driver.find_element_by_id(text).click()
    time.sleep(12)
    driver.quit()

def Locate_by_XPath(name, text):
    driver = webdriver.Chrome('C:/Users/smile/Documents/GitHub/chromedriver.exe')
    driver.get("https://ETF.com/" + name)
    driver.find_element_by_link_text("Fund Home Page").click()
    driver.find_element_by_xpath(text).click()
    time.sleep(12)
    driver.quit()
    
def Specific_for_VanEck(typ, name):
    driver = webdriver.Chrome('C:/Users/smile/Documents/GitHub/chromedriver.exe')
    driver.get("https://www.vaneck.com/etf/" + typ + "/" + name + "/documents/")
    #driver.get("https://www.vaneck.com/row/?country=row&vecs=true")
    driver.switch_to_frame('selector') 
    time.sleep(3)
    driver.find_element_by_id("retail").click() 
    driver.find_element_by_id("btnAgree").click()
    #driver.get("https://www.vaneck.com/etf/equity/" + name + "/documents/")
    driver.find_element_by_link_text("NAV & Premium/Discount History").click()
    time.sleep(12)
    driver.quit()

In [245]:
#classified by institution (different website)
ETFcom = { "Invesco": ['PZD', 'PXI', 'PXE', 'PXJ', 'PBD', 'RYE', 'PSCE', 'TAN', 'PBW'], 
          "SPDR": ['XLE'], "ProShares": ['DIG', 'DUG', 'DDG'], "Tortoise": ['TPYP'], 
          "VanEck1": ['KOL', 'GEX', 'CRAK', 'OIH', 'FRAK', 'NLR'], "VanEck2": ['YMLI', 'YMLP'],
         "iShares": ['ICLN', 'IXC', 'FILL', 'IYE', 'IEO', 'IEZ'],
         } 

In [171]:
#download as excel file (wait for processing)

#'DIG', 'DUG', 'DDG'
for ETF in ETFcom["ProShares"]:
    Locate_by_linkname(ETF, "NAV History")

#'PZD', 'PXI', 'PXE', 'PXJ', 'PBD', 'RYE', 'PSCE', 'TAN', 'PBW'
for ETF in ETFcom["Invesco"]:
    Locate_by_ID(ETF, "downloadNavHistory")

#'XLE'   
#for ETF in ETFcom["SPDR"]:
Locate_by_XPath('XLE', "//a[contains(text(),'Most Recent NAV / NAV History')]")

#'ICLN', 'IXC', 'FILL', 'IYE', 'IEO', 'IEZ'
for ETF in ETFcom["iShares"]:
    Locate_by_linkname(ETF, "Download")
    
#equity: 'KOL', 'GEX',  'CRAK', 'OIH', 'FRAK', 'NLR'
#income: 'YMLI', 'YMLP',
for ETF in ETFcom["VanEck1"]:
    Specific_for_VanEck("equity", ETF)
for ETF in ETFcom["VanEck2"]:
    Specific_for_VanEck("income", ETF)

In [157]:
#'TPYP'
r = requests.get("http://etp.tortoiseadvisors.com/tpyp/premium-discount/")
if r.status_code == requests.codes.ok:
    soup = BeautifulSoup(r.text, 'html.parser')
tags = soup.find_all('table')
df = pd.read_html(str(tags[0]))
dff = pd.DataFrame(df[0])
a = dff[0:781] #date filter
TPYP = a.iloc[:,[0,2]] #keep date & NAV
TPYP.head()

,Date,NAV
0,3/15/2019,$23.79
1,3/14/2019,$23.87
2,3/13/2019,$23.89
3,3/12/2019,$23.80
4,3/11/2019,$23.68
